
# 08 — Agentic RAG  
Planner → Retriever → Answer Agent using LangChain

This notebook provides a **minimal runnable Agentic RAG pipeline**:
1. Planner Agent  
2. Retriever Agent  
3. Answer Agent  
4. Combined workflow (non-LangGraph, linear version)


## 0. Setup

In [ ]:

!pip install langchain langchain-openai chromadb langchain-community --quiet

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

llm = ChatOpenAI(model="gpt-4o-mini")
emb = OpenAIEmbeddings()


## 1. Sample corpus

In [ ]:

import os
os.makedirs("agentic_rag_data", exist_ok=True)

with open("agentic_rag_data/corpus.txt", "w") as f:
    f.write("RAG retrieves documents to help the LLM answer questions.\n")
    f.write("Agents can plan tasks, call tools, and manage reasoning steps.\n")
    f.write("Combining RAG with agents improves structure and reliability.\n")

loader = TextLoader("agentic_rag_data/corpus.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(docs)

db = Chroma.from_documents(chunks, emb)
retriever = db.as_retriever(search_kwargs={"k": 3})


## 2. Planner Agent

In [ ]:

def planner_agent(question: str) -> str:
    prompt = f"""You are a Planner Agent.
Break the user's question into a short retrieval plan.

Question: {question}

Return a 1–2 sentence plan.
"""
    return llm.invoke(prompt).content

print(planner_agent("How do RAG and agents work together?"))


## 3. Retriever Agent

In [ ]:

def retriever_agent(plan: str) -> str:
    docs = retriever.get_relevant_documents(plan)
    return "\n".join(d.page_content for d in docs)

print(retriever_agent("Plan: retrieve RAG + agent concepts"))


## 4. Answer Agent

In [ ]:

def answer_agent(question: str, context: str) -> str:
    prompt = f"""You are an Answer Agent.
Use ONLY the context to answer the question.

Context:
{context}

Question: {question}

Provide a concise, structured explanation.
"""
    return llm.invoke(prompt).content

ctx = retriever_agent("Retrieve RAG + agent concepts")
print(answer_agent("How do RAG and agents complement each other?", ctx))


## 5. Combined Agentic RAG Pipeline

In [ ]:

def agentic_rag(question: str):
    plan = planner_agent(question)
    ctx = retriever_agent(plan)
    ans = answer_agent(question, ctx)
    return {
        "plan": plan,
        "context": ctx,
        "answer": ans
    }

out = agentic_rag("Explain the role of agents in a RAG system.")
out



## End of Notebook 08 — Agentic RAG  
You now have a minimal 3-agent RAG system ready for LangGraph expansion.
